# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [1]:
# Сделаем необходимые импорты
import torch
import numpy as np

from torch import nn
import torchvision

In [2]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
# возьму набор CIFAR-10, т.к. комп дохленький

import torchvision.transforms as transforms
from sklearn.preprocessing import label_binarize

train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=False)

#преобразовываем форму изображений и нормируем их
smpls_train = train_dataset.data.shape[0]
X_train = np.reshape(train_dataset.data,(smpls_train,3072))/255

#переводим метки в векторы
y_train = label_binarize(train_dataset.targets,classes = [x for x in range(10)])
del train_dataset


test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=False,  
                                             transform=transforms.ToTensor(), 
                                             download=False)

#преобразовываем форму изображений и нормируем их
smpls_test = test_dataset.data.shape[0]
X_test = np.reshape(test_dataset.data,(smpls_test,3072))/255

#переводим метки в векторы
y_test = label_binarize(test_dataset.targets,classes = [x for x in range(10)])

classes = test_dataset.classes #названия меток
del test_dataset

#ОБратный перевод данных в тензоры
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

torch.Size([50000, 3072]) torch.Size([50000, 10]) torch.Size([10000, 3072]) torch.Size([10000, 10])


In [9]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации
from tqdm import tqdm

import torch.nn.functional as fnc
import torch.optim as opt

#4-слойный перцептрон
class Model_4l(nn.Module):
    def __init__(self,lays):
        super(Model_4l, self).__init__()
        self.lay12=nn.Linear(lays[0],lays[1]).to(device)   
        self.lay23=nn.Linear(lays[1],lays[2]).to(device)   
        self.lay34=nn.Linear(lays[2],lays[3]).to(device)
        
        self.opt=opt.SGD(self.parameters(), lr=0.001, momentum=0.9) #оптимизатор
        self.criterion=nn.CrossEntropyLoss() #функция потерь (отриц. логарифм правдоподобия)

    def forward(self, x):
        x = fnc.leaky_relu(self.lay12(x), 0.1)
        x = fnc.leaky_relu(self.lay23(x), 0.1)
        return fnc.softmax(self.lay34(x))


#5-слойный перцептрон
class Model_5l(nn.Module):
    def __init__(self,lays):
        super(Model_5l, self).__init__()
        self.lay12=nn.Linear(lays[0],lays[1]).to(device)   
        self.lay23=nn.Linear(lays[1],lays[2]).to(device)   
        self.lay34=nn.Linear(lays[2],lays[3]).to(device)
        self.lay45=nn.Linear(lays[3],lays[4]).to(device)
        
        self.opt=opt.SGD(self.parameters(), lr=0.001, momentum=0.9) #оптимизатор
        self.criterion=nn.CrossEntropyLoss() #функция потерь (отриц. логарифм правдоподобия)

    def forward(self, x):
        x = fnc.leaky_relu(self.lay12(x), 0.1)
        x = fnc.leaky_relu(self.lay23(x), 0.1)
        x = fnc.leaky_relu(self.lay34(x), 0.1)
        return fnc.softmax(self.lay45(x))


#6-слойный перцептрон
class Model_6l(nn.Module):
    def __init__(self,lays):
        super(Model_6l, self).__init__()
        self.lay12=nn.Linear(lays[0],lays[1]).to(device)   
        self.lay23=nn.Linear(lays[1],lays[2]).to(device)   
        self.lay34=nn.Linear(lays[2],lays[3]).to(device)
        self.lay45=nn.Linear(lays[3],lays[4]).to(device)
        self.lay56=nn.Linear(lays[4],lays[5]).to(device)
        
        self.opt=opt.SGD(self.parameters(), lr=0.001, momentum=0.9) #оптимизатор
        self.criterion=nn.CrossEntropyLoss() #функция потерь (отриц. логарифм правдоподобия)

    def forward(self, x):
        x = fnc.leaky_relu(self.lay12(x), 0.1)
        x = fnc.leaky_relu(self.lay23(x), 0.1)
        x = fnc.leaky_relu(self.lay34(x), 0.1)
        x = fnc.leaky_relu(self.lay45(x), 0.1)
        return fnc.softmax(self.lay56(x))

In [4]:
# Запустить обучение (по аналогии с тем, что делали на паре)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

size_in = 3072
size_out = 10

In [6]:
#обучим 4-слойные НС
errs_4l = []
mdls_4l = []
err_best = np.Inf #наименьшее достигнутое значение ошибки на тестовой выборке

#обучим 4-слойные сети
l2 = [768, 384, 192, 128]
l3 = [192, 128, 64, 32]

#определим размер пакета
batch_size = 64
batches = int(np.ceil(smpls_train/batch_size))

for l2_ in l2:
    for l3_ in l3:
        net = Model_4l([size_in, l2_, l3_, size_out])

        for epoch in tqdm(range(20)):  
            running_loss = 0.0
            for i in range(batches):
                if i+1 != batches:
                    inputs = X_train[i*batch_size:(i+1)*batch_size, :]
                    labels = y_train[i*batch_size:(i+1)*batch_size, :]
                else:
                    inputs1 = X_train[i*batch_size:smpls_train, :]
                    labels1 = y_train[i*batch_size:smpls_train, :]
                    
                    resid = batch_size - (smpls_train - i*batch_size)
                    
                    inputs2 = X_train[0:resid, :]
                    labels2 = y_train[0:resid, :]
                    
                    inputs = torch.cat((inputs1,inputs2))
                    labels = torch.cat((labels1,labels2))
                    
                    del inputs1, labels1, inputs2, labels2

                # обнуляем градиент
                net.opt.zero_grad()
                
                #inputs = inputs.reshape((inputs.shape[0],3072)) #переводим образцы в векторы
                #inputs = inputs/255 #и нормируем

                outputs = net(inputs)
                loss = net.criterion(outputs, labels)
                loss.backward()
                net.opt.step()

                # выводим статистику о процессе обучения
                running_loss += loss.item()
                if i % 300 == 0:    # печатаем каждые 300 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000)) # откуда 2000?
                    running_loss = 0.0
                
            
        
        #вычисление ошибки на тестовом наборе данных
        y_pred = net(X_test)
        loss_test = net.criterion(y_pred, y_test).item()
        
        if loss_test<err_best: #сохраняем только лучшие сети на данный момент
            errs_4l.append(loss_test)
            mdls_4l.append(net)
            err_best = loss_test

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4048/27334768.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return fnc.softmax(self.lay34(x))


[1,     1] loss: 0.001
[1,   301] loss: 0.345
[1,   601] loss: 0.345


  5%|████▏                                                                              | 1/20 [00:49<15:43, 49.66s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.344
[2,   601] loss: 0.344


 10%|████████▎                                                                          | 2/20 [01:32<13:39, 45.51s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [02:56<00:18,  9.43s/it]

[19,     1] loss: 0.001
[19,   301] loss: 0.318
[19,   601] loss: 0.317


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:06<00:09,  9.40s/it]

[20,     1] loss: 0.001
[20,   301] loss: 0.318
[20,   601] loss: 0.317


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:15<00:00,  9.76s/it]


In [7]:
#обучим 5-слойные НС
errs_5l = []
mdls_5l = []
err_best = np.Inf #наименьшее достигнутое значение ошибки на тестовой выборке

#обучим 5-слойные сети
l2 = [576, 320, 128]
l3 = [224, 160, 128]
l4 = [96, 64]

#определим размер пакета
batch_size = 64
batches = int(np.ceil(smpls_train/batch_size))

for l2_ in l2:
    for l3_ in l3:
        for l4_ in l4:
            net = Model_5l([size_in, l2_, l3_, l4_, size_out])

            for epoch in tqdm(range(20)):  
                running_loss = 0.0
                for i in range(batches):
                    if i+1 != batches:
                        inputs = X_train[i*batch_size:(i+1)*batch_size, :]
                        labels = y_train[i*batch_size:(i+1)*batch_size, :]
                    else:
                        inputs1 = X_train[i*batch_size:smpls_train, :]
                        labels1 = y_train[i*batch_size:smpls_train, :]

                        resid = batch_size - (smpls_train - i*batch_size)

                        inputs2 = X_train[0:resid, :]
                        labels2 = y_train[0:resid, :]

                        inputs = torch.cat((inputs1,inputs2))
                        labels = torch.cat((labels1,labels2))

                        del inputs1, labels1, inputs2, labels2

                    # обнуляем градиент
                    net.opt.zero_grad()

                    #inputs = inputs.reshape((inputs.shape[0],3072)) #переводим образцы в векторы
                    #inputs = inputs/255 #и нормируем

                    outputs = net(inputs)
                    loss = net.criterion(outputs, labels)
                    loss.backward()
                    net.opt.step()

                    # выводим статистику о процессе обучения
                    running_loss += loss.item()
                    if i % 300 == 0:    # печатаем каждые 300 mini-batches
                        print('[%d, %5d] loss: %.3f' %
                              (epoch + 1, i + 1, running_loss / 2000)) # откуда 2000?
                        running_loss = 0.0



            #вычисление ошибки на тестовом наборе данных
            y_pred = net(X_test)
            loss_test = net.criterion(y_pred, y_test).item()

            if loss_test<err_best: #сохраняем только лучшие сети на данный момент
                errs_5l.append(loss_test)
                mdls_5l.append(net)
                err_best = loss_test

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4048/27334768.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return fnc.softmax(self.lay45(x))


[1,     1] loss: 0.001
[1,   301] loss: 0.345
[1,   601] loss: 0.345


  5%|████▏                                                                              | 1/20 [00:37<12:01, 37.99s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.345
[2,   601] loss: 0.345


 10%|████████▎                                                                          | 2/20 [01:14<11:12, 37.34s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [02:45<00:18,  9.21s/it]

[19,     1] loss: 0.001
[19,   301] loss: 0.324
[19,   601] loss: 0.324


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [02:54<00:09,  9.18s/it]

[20,     1] loss: 0.001
[20,   301] loss: 0.324
[20,   601] loss: 0.323


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:03<00:00,  9.18s/it]


In [10]:
#обучим 6-слойные НС
errs_6l = []
mdls_6l = []
err_best = np.Inf #наименьшее достигнутое значение ошибки на тестовой выборке

#обучим 5-слойные сети
l2 = [448, 320]
l3 = [256, 160]
l4 = [128, 112]
l5 = [96, 48]

#определим размер пакета
batch_size = 64
batches = int(np.ceil(smpls_train/batch_size))

for l2_ in l2:
    for l3_ in l3:
        for l4_ in l4:
            for l5_ in l5:
                net = Model_6l([size_in, l2_, l3_, l4_, l5_, size_out])

                for epoch in tqdm(range(20)):  
                    running_loss = 0.0
                    for i in range(batches):
                        if i+1 != batches:
                            inputs = X_train[i*batch_size:(i+1)*batch_size, :]
                            labels = y_train[i*batch_size:(i+1)*batch_size, :]
                        else:
                            inputs1 = X_train[i*batch_size:smpls_train, :]
                            labels1 = y_train[i*batch_size:smpls_train, :]

                            resid = batch_size - (smpls_train - i*batch_size)

                            inputs2 = X_train[0:resid, :]
                            labels2 = y_train[0:resid, :]

                            inputs = torch.cat((inputs1,inputs2))
                            labels = torch.cat((labels1,labels2))

                            del inputs1, labels1, inputs2, labels2

                        # обнуляем градиент
                        net.opt.zero_grad()

                        outputs = net(inputs)
                        loss = net.criterion(outputs, labels)
                        loss.backward()
                        net.opt.step()

                        # выводим статистику о процессе обучения
                        running_loss += loss.item()
                        if i % 300 == 0:    # печатаем каждые 300 mini-batches
                            print('[%d, %5d] loss: %.3f' %
                                  (epoch + 1, i + 1, running_loss / 2000)) # откуда 2000?
                            running_loss = 0.0



                #вычисление ошибки на тестовом наборе данных
                y_pred = net(X_test)
                loss_test = net.criterion(y_pred, y_test).item()

                if loss_test<err_best: #сохраняем только лучшие сети на данный момент
                    errs_6l.append(loss_test)
                    mdls_6l.append(net)
                    err_best = loss_test

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4048/3208744062.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return fnc.softmax(self.lay56(x))


[1,     1] loss: 0.001
[1,   301] loss: 0.345
[1,   601] loss: 0.345


  5%|████▏                                                                              | 1/20 [00:29<09:13, 29.14s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.345
[2,   601] loss: 0.345


 10%|████████▎                                                                          | 2/20 [00:57<08:36, 28.67s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [09:18<00:47, 23.69s/it]

[19,     1] loss: 0.001
[19,   301] loss: 0.340
[19,   601] loss: 0.339


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [09:41<00:23, 23.49s/it]

[20,     1] loss: 0.001
[20,   301] loss: 0.338
[20,   601] loss: 0.338


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [10:07<00:00, 30.36s/it]


In [12]:
#Выведем результаты обучения всех сетей (лучшие)
errs_4l = np.array(errs_4l)
nn_4l_i = errs_4l.argmin()

errs_5l = np.array(errs_5l)
nn_5l_i = errs_5l.argmin()

errs_6l = np.array(errs_6l)
nn_6l_i = errs_6l.argmin()

print('4-слойные сети: наименьшая ошибка: {}, \nСеть:\n{}'.format(errs_4l[nn_4l_i],mdls_4l[nn_4l_i]),end = '\n\n')
print('5-слойные сети: наименьшая ошибка: {}, \nСеть:\n{}'.format(errs_5l[nn_5l_i],mdls_5l[nn_5l_i]),end = '\n\n')
print('6-слойные сети: наименьшая ошибка: {}, \nСеть:\n{}'.format(errs_6l[nn_6l_i],mdls_6l[nn_6l_i]),end = '\n\n')

4-слойные сети: наименьшая ошибка: 2.1084680557250977, 
Сеть:
Model_4l(
  (lay12): Linear(in_features=3072, out_features=768, bias=True)
  (lay23): Linear(in_features=768, out_features=128, bias=True)
  (lay34): Linear(in_features=128, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)

5-слойные сети: наименьшая ошибка: 2.1301937103271484, 
Сеть:
Model_5l(
  (lay12): Linear(in_features=3072, out_features=128, bias=True)
  (lay23): Linear(in_features=128, out_features=160, bias=True)
  (lay34): Linear(in_features=160, out_features=96, bias=True)
  (lay45): Linear(in_features=96, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)

6-слойные сети: наименьшая ошибка: 2.177771806716919, 
Сеть:
Model_6l(
  (lay12): Linear(in_features=3072, out_features=448, bias=True)
  (lay23): Linear(in_features=448, out_features=256, bias=True)
  (lay34): Linear(in_features=256, out_features=112, bias=True)
  (lay45): Linear(in_features=112, out_features=96, bias=True)
  (lay56)

In [14]:
#сохраним лучшую модель
best_mdl = mdls_4l[nn_4l_i]

checkpoint={
    'state_dict': best_mdl.state_dict(),
    'optimizer': best_mdl.opt.state_dict()
}
torch.save(checkpoint,'best_mdl.pt')

#ЗАГРУЗКА МОДЕЛИ
'''
best_mdl2 = Model_4l([3072, 768, 128, 10])
checkpoint=torch.load('best_mdl.pt')
best_mdl2.load_state_dict(checkpoint['state_dict'])
best_mdl2.opt.load_state_dict(checkpoint['optimizer'])#ОБЯЗАТЕЛЬНО!
best_mdl2.opt.zero_grad()
'''

"\nbest_mdl2 = Model_4l([3072, 768, 128, 10])\ncheckpoint=torch.load('best_mdl.pt')\nbest_mdl2.load_state_dict(checkpoint['state_dict'])\nbest_mdl2.opt.load_state_dict(checkpoint['optimizer'])#ОБЯЗАТЕЛЬНО!\nbest_mdl2.opt.zero_grad()\n"